In [1]:
import os
import sys
sys.path.append('common')
import util, audio_preprocessing

import shutil
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint, Callback, TensorBoard
from matplotlib import pyplot as plt
%matplotlib inline
from cyclic_lr import CyclicLR

from dataset import SingleDataset, MultiDataset
from model import model_mlt_cnn_alexnet

TRY = 'MIX9'
BASE = 'MIX6'
LOG_DIR = './%slog' % TRY
SCD_PREFIX = 'scd_'
USD_PREFIX = 'usd_'

batch_size = 32
epochs = 20

scd = SingleDataset(SCD_PREFIX,
                   util.load_txt_list(os.path.join('.', SCD_PREFIX+'classes.txt')),
                   batch_size)
usd = SingleDataset(USD_PREFIX,
                   ['air_conditioner',  'car_horn', 'children_playing', 'dog_bark', 'drilling',
                    'engine_idling', 'gun_shot','jackhammer', 'siren', 'street_music'],
                   batch_size)

Using TensorFlow backend.


In [2]:
# Clean up log
shutil.rmtree(LOG_DIR, ignore_errors=True)

In [3]:
# Fine tune for scd
multi = MultiDataset([scd, usd], [1.0, 0.0])

model = model_mlt_cnn_alexnet(multi.input_shape(), multi.ys_classes(), freeze_mode=1)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
model.load_weights('model%s.h5' % BASE) ### Starting point

callbacks = [
    CyclicLR(base_lr=0.00007, max_lr=0.0007, step_size=multi.train_steps_per_epoch, mode='triangular'),
    ModelCheckpoint('model%s.h5' % TRY,
                monitor='val_output_1_acc',
                verbose=1,
                save_best_only=True,
                save_weights_only=True),
     keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
]
model.fit_generator(multi.train_generator,
                    steps_per_epoch=multi.train_steps_per_epoch,
                    epochs=epochs,
                    validation_data=multi.valid_generator, 
                    validation_steps=multi.valid_steps_per_epoch,
                    callbacks=callbacks)

multi.evaluate_by_datasets(model)

# Fine tune for usd
multi = MultiDataset([scd, usd], [0.0, 1.0])

model = model_mlt_cnn_alexnet(multi.input_shape(), multi.ys_classes(), freeze_mode=2)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
model.load_weights('model%s.h5' % TRY)

callbacks = [
    CyclicLR(base_lr=0.00007, max_lr=0.0007, step_size=multi.train_steps_per_epoch, mode='triangular'),
    ModelCheckpoint('model%s.h5' % TRY,
                monitor='val_output_2_acc',
                verbose=1,
                save_best_only=True,
                save_weights_only=True),
     keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)
]
model.fit_generator(multi.train_generator,
                    steps_per_epoch=multi.train_steps_per_epoch,
                    epochs=epochs,
                    validation_data=multi.valid_generator, 
                    validation_steps=multi.valid_steps_per_epoch,
                    callbacks=callbacks)

multi.evaluate_by_datasets(model)

model_mlt_cnn_alexnet: freeze other than output_1.
Epoch 1/20
1594/1596 [============================>.] - ETA: 0s - loss: 1.7466 - output_1_loss: 1.7599 - output_2_loss: -0.0133 - output_1_acc: 0.7334 - output_2_acc: 0.1444
Epoch 00001: val_output_1_acc improved from -inf to 0.93182, saving model to modelMIX9.h5
1596/1596 [==============================] - 72s 45ms/step - loss: 1.7468 - output_1_loss: 1.7600 - output_2_loss: -0.0132 - output_1_acc: 0.7334 - output_2_acc: 0.1444 - val_loss: 0.2562 - val_output_1_loss: 0.2695 - val_output_2_loss: -0.0133 - val_output_1_acc: 0.9318 - val_output_2_acc: 0.0922
Epoch 2/20
1594/1596 [============================>.] - ETA: 0s - loss: 1.7594 - output_1_loss: 1.7646 - output_2_loss: -0.0052 - output_1_acc: 0.7337 - output_2_acc: 0.1414
Epoch 00002: val_output_1_acc did not improve
1596/1596 [==============================] - 71s 45ms/step - loss: 1.7588 - output_1_loss: 1.7644 - output_2_loss: -0.0056 - output_1_acc: 0.7337 - output_2_acc: 0.14

1595/1596 [============================>.] - ETA: 0s - loss: 1.7460 - output_1_loss: 1.7510 - output_2_loss: -0.0050 - output_1_acc: 0.7316 - output_2_acc: 0.1448
Epoch 00016: val_output_1_acc did not improve
1596/1596 [==============================] - 71s 45ms/step - loss: 1.7458 - output_1_loss: 1.7509 - output_2_loss: -0.0051 - output_1_acc: 0.7316 - output_2_acc: 0.1448 - val_loss: 0.2462 - val_output_1_loss: 0.2620 - val_output_2_loss: -0.0158 - val_output_1_acc: 0.9316 - val_output_2_acc: 0.0909
Epoch 17/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.7336 - output_1_loss: 1.7477 - output_2_loss: -0.0141 - output_1_acc: 0.7343 - output_2_acc: 0.1409
Epoch 00017: val_output_1_acc did not improve
1596/1596 [==============================] - 71s 45ms/step - loss: 1.7334 - output_1_loss: 1.7477 - output_2_loss: -0.0143 - output_1_acc: 0.7343 - output_2_acc: 0.1408 - val_loss: 0.2480 - val_output_1_loss: 0.2630 - val_output_2_loss: -0.0150 - val_output_1_acc: 0.9325

Epoch 11/20
1594/1596 [============================>.] - ETA: 0s - loss: 1.3990 - output_1_loss: -0.0036 - output_2_loss: 1.4026 - output_1_acc: 0.0413 - output_2_acc: 0.6421
Epoch 00011: val_output_2_acc did not improve
1596/1596 [==============================] - 72s 45ms/step - loss: 1.3995 - output_1_loss: -0.0033 - output_2_loss: 1.4029 - output_1_acc: 0.0413 - output_2_acc: 0.6419 - val_loss: 1.4287 - val_output_1_loss: -3.5642e-04 - val_output_2_loss: 1.4291 - val_output_1_acc: 0.0288 - val_output_2_acc: 0.5312
Epoch 12/20
1595/1596 [============================>.] - ETA: 0s - loss: 1.4172 - output_1_loss: 0.0111 - output_2_loss: 1.4062 - output_1_acc: 0.0411 - output_2_acc: 0.6422
Epoch 00012: val_output_2_acc did not improve
1596/1596 [==============================] - 72s 45ms/step - loss: 1.4172 - output_1_loss: 0.0111 - output_2_loss: 1.4061 - output_1_acc: 0.0411 - output_2_acc: 0.6422 - val_loss: 1.4173 - val_output_1_loss: 0.0076 - val_output_2_loss: 1.4098 - val_output_

[0.9349808767284495, 0.5292929292929293]

## Evaluate per dataset

In [4]:
model.load_weights('model%s.h5' % TRY)
multi.evaluate_by_datasets(model)

Evaluating scd_dataset
6798/6798 [==============================] - 10s 1ms/step
 = 0.9349808767284495
Evaluating usd_dataset
990/990 [==============================] - 1s 2ms/step
 = 0.5434343434343434
Accuracy with scd_dataset = 0.934981
Accuracy with usd_dataset = 0.543434


[0.9349808767284495, 0.5434343434343434]

## Evaluate by mixed dataset

In [5]:
model.load_weights('model%s.h5' % TRY)
multi = MultiDataset([scd, usd], [0.6, 0.4], mix_randomness=0.0)
results = model.evaluate_generator(multi.valid_generator, steps=multi.valid_steps_per_epoch)
print(model.metrics_names, results)

['loss', 'output_1_loss', 'output_2_loss', 'output_1_acc', 'output_2_acc'] [0.8975593357958745, 0.26930842500272806, 0.6282509113889155, 0.8967642526964561, 0.4619928094504366]
